**Goal** : Concatenate all dataframes related to DPCFam MC properties into a consistent one.

In [1]:
# 0. Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# 1. Paths to files
# Root Path
root_path = "/u/mdmc/enyanduk/internship_areasciencepark/Dataframes/DPCFam/"
# Files
# 1. MC LIST
mc_list = root_path + "mclist.csv"
df1 = pd.read_csv(mc_list)
# 2. SEQUENCE INFORMATION
sequence_information = root_path + "sequence_information.csv"
df2 = pd.read_csv(sequence_information)
# 3. PFAM COMPARISON
pfam_comparison = root_path + "pfam_comparison.csv"
df3 = pd.read_csv(pfam_comparison)
# 4. LC REGIONS
lc_regions = root_path + "lcregions.csv"
df4 = pd.read_csv(lc_regions)
# 5. CC REGIONS
cc_regions = root_path + "ccregions.csv"
df5 = pd.read_csv(cc_regions)
# 6. DIS REGIONS
dis_regions = root_path + "disregions.csv"
df6 = pd.read_csv(dis_regions)
# 7. TM REGIONS
tm_regions = root_path + "tmregions.csv"
df7 = pd.read_csv(tm_regions)

In [3]:
# List of dataframes
dfs = [df1, df2, df3, df4, df5, df6, df7]

In [4]:
# Merge all dataframes:
df = (
    pd.concat(
        [d.set_index("MCID") for d in dfs],
        axis=1,
        join="outer"  # keeps all MCIDs
    )
    .reset_index()
 )

In [5]:
df.head()

,MCID,seed_size,average_lenght,std_average_length,DA,DAC,%DA,%DAC,%DACF,%DACFA,AvOv,fred,fext,LABEL,pfam_seqs,%LC,%CC,%DIS,TM
0,1,17931,185.6810,28.76910,PF13614,CCL0023,0.442271,0.855137,0.989872,0.999022,0.808203,0.058723,0.134096,equivalent,6332,0.047194,0.000000,0.184394,0.009800
1,4,617,59.9109,6.06690,PF03600,CCL0182,0.628415,0.704918,0.970856,1.000000,0.075365,0.917423,0.699776,shifted,345,0.049879,0.000000,0.018654,1.261900
2,15,139,81.2086,5.05150,UNK,UNK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,131,0.049560,0.001799,0.138066,0.029126
3,19,120,71.5667,7.69711,PF11915,PF11915,0.940678,0.940678,1.000000,1.000000,0.136637,0.774788,0.722817,shifted,111,0.088535,0.022461,0.048627,1.689660
4,21,937,91.1974,7.69776,PF01012,CCL0039,0.988998,0.988998,1.000000,1.000000,0.345184,0.591957,0.278153,shifted,809,0.026462,0.000000,0.243835,0.000000


In [6]:
# Drop some useless columns
df = df.drop(columns=["DAC", "%DAC", "%DACF", "%DACFA", "fred", "fext"])

In [7]:
# New dataframe
df.head()

,MCID,seed_size,average_lenght,std_average_length,DA,%DA,AvOv,LABEL,pfam_seqs,%LC,%CC,%DIS,TM
0,1,17931,185.6810,28.76910,PF13614,0.442271,0.808203,equivalent,6332,0.047194,0.000000,0.184394,0.009800
1,4,617,59.9109,6.06690,PF03600,0.628415,0.075365,shifted,345,0.049879,0.000000,0.018654,1.261900
2,15,139,81.2086,5.05150,UNK,NaN,NaN,NaN,131,0.049560,0.001799,0.138066,0.029126
3,19,120,71.5667,7.69711,PF11915,0.940678,0.136637,shifted,111,0.088535,0.022461,0.048627,1.689660
4,21,937,91.1974,7.69776,PF01012,0.988998,0.345184,shifted,809,0.026462,0.000000,0.243835,0.000000


In [8]:
# Rearrange columns properly
df = df[[
    "MCID", "seed_size", "average_lenght", "std_average_length", "%LC", "%CC", "%DIS", "TM", "DA", "%DA", "pfam_seqs", "AvOv", "LABEL"
    ]]
df.head()

,MCID,seed_size,average_lenght,std_average_length,%LC,%CC,%DIS,TM,DA,%DA,pfam_seqs,AvOv,LABEL
0,1,17931,185.6810,28.76910,0.047194,0.000000,0.184394,0.009800,PF13614,0.442271,6332,0.808203,equivalent
1,4,617,59.9109,6.06690,0.049879,0.000000,0.018654,1.261900,PF03600,0.628415,345,0.075365,shifted
2,15,139,81.2086,5.05150,0.049560,0.001799,0.138066,0.029126,UNK,NaN,131,NaN,NaN
3,19,120,71.5667,7.69711,0.088535,0.022461,0.048627,1.689660,PF11915,0.940678,111,0.136637,shifted
4,21,937,91.1974,7.69776,0.026462,0.000000,0.243835,0.000000,PF01012,0.988998,809,0.345184,shifted


In [9]:
# We rename columns properly to avoid conflicts in PostgreSQL:
df = df.rename(columns={
    "MCID": "mcid",
    "seed_size": "size_uniref50",
    "average_lenght": "avg_len",
    "std_average_length": "std_avg_len",
    "%LC": "lc_percent",
    "%CC": "cc_percent",
    "%DIS": "dis_percent",
    "TM": "tm",
    "DA": "pfam_da",
    "%DA": "da_percent",
    "pfam_seqs": "size_pfam",
    "AvOv": "avg_ov_percent",
    "LABEL": "overlap_label"
})
df.head()

,mcid,size_uniref50,avg_len,std_avg_len,lc_percent,cc_percent,dis_percent,tm,pfam_da,da_percent,size_pfam,avg_ov_percent,overlap_label
0,1,17931,185.6810,28.76910,0.047194,0.000000,0.184394,0.009800,PF13614,0.442271,6332,0.808203,equivalent
1,4,617,59.9109,6.06690,0.049879,0.000000,0.018654,1.261900,PF03600,0.628415,345,0.075365,shifted
2,15,139,81.2086,5.05150,0.049560,0.001799,0.138066,0.029126,UNK,NaN,131,NaN,NaN
3,19,120,71.5667,7.69711,0.088535,0.022461,0.048627,1.689660,PF11915,0.940678,111,0.136637,shifted
4,21,937,91.1974,7.69776,0.026462,0.000000,0.243835,0.000000,PF01012,0.988998,809,0.345184,shifted


In [10]:
# We perform some meaningful transformations to make the data more interpretable:
# T1: Rewrite each ID in mcid column as MCID: e.g.: 1 -> MC1
df["mcid"] = df["mcid"].apply(lambda x: f"MC{x}")
# T2: Columns size_uniref50 and size_pfam store integer values. We convert them to int type:
df["size_uniref50"] = df["size_uniref50"].astype(int)
df["size_pfam"] = df["size_pfam"].astype(int)
# T3: For each value != NaN in columns (lc_percent, cc_percent, dis_percent, da_percent, avg_ov_percent), multiply it by 100 and round it (2 digits after ,)
df[["lc_percent", "cc_percent", "dis_percent", "da_percent", "avg_ov_percent"]] = df[["lc_percent", "cc_percent", "dis_percent", "da_percent", "avg_ov_percent"]].fillna(0).multiply(100).round(2)
# T4: Round columns avg_len, std_avg_len and tm to 2 digits:
df[["avg_len", "std_avg_len", "tm"]] = df[["avg_len", "std_avg_len", "tm"]].round(2)
# T5: In column pfam_da: replace UNK by UNKNOWN, in overlap_label: replace NaN by NONE
df["pfam_da"] = df["pfam_da"].replace("UNK", "UNKNOWN")
df["overlap_label"] = df["overlap_label"].fillna("NONE")
df.head()

,mcid,size_uniref50,avg_len,std_avg_len,lc_percent,cc_percent,dis_percent,tm,pfam_da,da_percent,size_pfam,avg_ov_percent,overlap_label
0,MC1,17931,185.68,28.77,4.72,0.00,18.44,0.01,PF13614,44.23,6332,80.82,equivalent
1,MC4,617,59.91,6.07,4.99,0.00,1.87,1.26,PF03600,62.84,345,7.54,shifted
2,MC15,139,81.21,5.05,4.96,0.18,13.81,0.03,UNKNOWN,0.00,131,0.00,NONE
3,MC19,120,71.57,7.70,8.85,2.25,4.86,1.69,PF11915,94.07,111,13.66,shifted
4,MC21,937,91.20,7.70,2.65,0.00,24.38,0.00,PF01012,98.90,809,34.52,shifted


In [11]:
# Infos
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 46828 entries, 0 to 46827
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   mcid            46828 non-null  str    
 1   size_uniref50   46828 non-null  int64  
 2   avg_len         46828 non-null  float64
 3   std_avg_len     46828 non-null  float64
 4   lc_percent      46828 non-null  float64
 5   cc_percent      46828 non-null  float64
 6   dis_percent     46828 non-null  float64
 7   tm              46828 non-null  float64
 8   pfam_da         46828 non-null  str    
 9   da_percent      46828 non-null  float64
 10  size_pfam       46828 non-null  int64  
 11  avg_ov_percent  46828 non-null  float64
 12  overlap_label   46828 non-null  str    
dtypes: float64(8), int64(2), str(3)
memory usage: 4.6 MB


In [12]:
# Statistics
df.describe().T

,count,mean,std,min,25%,50%,75%,max
size_uniref50,46828.0,328.904758,2146.797795,50.00,69.00,104.000,196.0000,131827.00
avg_len,46828.0,175.568361,162.883216,50.00,79.41,120.960,208.3250,3150.42
std_avg_len,46828.0,22.761171,25.212310,0.13,8.47,14.300,27.1700,544.91
lc_percent,46828.0,4.425717,4.629315,0.00,1.51,2.850,5.6000,49.97
cc_percent,46828.0,1.580738,7.391210,0.00,0.00,0.000,0.0200,86.34
dis_percent,46828.0,24.050744,14.235749,0.09,14.89,21.820,30.3200,92.85
tm,46828.0,0.348141,1.163270,0.00,0.00,0.000,0.0400,35.25
da_percent,46828.0,46.016375,43.294262,0.00,0.00,43.900,94.4400,100.00
size_pfam,46828.0,188.447339,1090.696865,1.00,37.00,62.000,123.0000,90231.00
avg_ov_percent,46828.0,30.051564,31.305588,0.00,0.00,21.335,49.9425,99.56


In [13]:
# Save to csv
final_target = root_path + "dpcfam_mcs_props.csv"
df.to_csv(final_target, index=False)